In [1]:
import time
from typing import Optional
import chipwhisperer as cw
from Crypto.Cipher import AES
from tqdm.notebook import tnrange

In [2]:
def configure_target(fpga_id='100t'):
    bitstreamfile = './hardware/victims/firmware/cw305_aes/cw305_top.bit'
    target = cw.target(None, cw.targets.CW305, force=True, bsfile=bitstreamfile)
    
    target.vccint_set(1.0)
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)
    target.pll.pll_outfreq_set(10E6, 1)

    target.clkusbautooff = True
    target.clksleeptime = 1

    return target


target = configure_target()

# Initialize the key and plaintext generator
ktp = cw.ktp.Basic()

# Lists to store keys and plaintexts
textin = []
keys = []
N = 1000  # Number of traces
traces = []

# Initialize cipher to verify DUT result
key, text = ktp.next()
cipher = AES.new(bytes(key), AES.MODE_ECB)
# Capture traces
for i in tnrange(N, desc='Capturing traces'):
    # Generate next key and plaintext pair
    key, text = ktp.next()
    textin.append(text)
    keys.append(key)
    
    # Encrypt using the CW305 target
    ciphertext = cw.encrypt(target, text, key)
    # Verify the encryption result against the expected value
    expected_ciphertext = cipher.encrypt(bytes(text))
    assert list(ciphertext) == list(expected_ciphertext), \
        f"Incorrect encryption result! Got {ciphertext} Exp {list(expected_ciphertext)}"

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

(ChipWhisperer Target ERROR|File CW305.py:365) target.REG_CRYPT_KEY unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:374) target.REG_CRYPT_TEXTIN unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:441) target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT

(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?
(ChipWhisperer Target ERROR|File CW305.py:383) target.REG_CRYPT_GO or target.REG_USER_LED unset. Have you given target a verilog defines file?

TypeError: 'NoneType' object is not iterable

In [1]:
import pyvisa
import time
from typing import Optional
import chipwhisperer as cw
from Crypto.Cipher import AES
from tqdm.notebook import tnrange

# Configure the FPGA target
def configure_target():
    bitstreamfile = './hardware/victims/firmware/cw305_aes/cw305_top.bit'
    target = cw.target(cw.targets.CW305, bsfile=bitstreamfile, force=True)
    '''
    bitstreamfile = './hardware/victims/firmware/cw305_aes/cw305_top.bit'
    target = cw.target(None, cw.targets.CW305, fpga_id="100t") # or 
    spi = target.spi_mode()
    spi.erase_chip() # erase full chip
    with open(bitstreamfile, "rb") as f:
        data = list(f.read())
        spi.program(data)
    '''
    target.vccint_set(1.0)
    target.pll.pll_enable_set(True)
    target.pll.pll_outenable_set(False, 0)
    target.pll.pll_outenable_set(True, 1)
    target.pll.pll_outenable_set(False, 2)
    target.pll.pll_outfreq_set(10E6, 1)

    target.clkusbautooff = True
    target.clksleeptime = 1

    return target

# Initialize the target
target = configure_target()

# Connect to the oscilloscope
rm = pyvisa.ResourceManager()
#oscilloscope = rm.open_resource('USB0::0x0957::0x179A::MY1234567::INSTR')
oscilloscope = rm.open_resource('USB0::0x1AB1::0x0515::MS5A221000657::INSTR')


# Configure oscilloscope settings
oscilloscope.write("*RST")  # Reset the oscilloscope to its default settings
oscilloscope.write(":CHAN1:COUP AC")
oscilloscope.write(':ACQ:SRAT 200e6')  # Set sampling rate to 200 MSa/s
oscilloscope.write(":TIMEBASE:MODE MAIN")  # Set the timebase mode to main (standard timebase)
oscilloscope.write(":TIMEBASE:RANGE 20e-3")  # Set the timebase range (time window) to 20 millisecond
oscilloscope.write(":TRIGGER:EDGE:SOURCE EXT")  # Set the trigger source to the external trigger input
oscilloscope.write(":TRIGGER:EDGE:LEVEL 1.0")  # Set the trigger level (voltage threshold) for the external trigger


AttributeError: type object 'CW305' has no attribute '_get_usart'

In [1]:
# Initialize the key and plaintext generator
ktp = cw.ktp.Basic()

# Lists to store keys and plaintexts
textin = []
keys = []
N = 1000  # Number of traces
traces = []
project_file = "projects/Tutorial_HW_CW305.cwp"
project = cw.create_project(project_file, overwrite=True)

# Function to capture a trace
def capture_trace():
    oscilloscope.write(":SINGLE")  # Set the oscilloscope to single trigger mode
    oscilloscope.query("*OPC?")  # Wait for the operation to complete
    trace_data = oscilloscope.query_binary_values(':WAV:DATA?', datatype='B')  # Fetch the captured data as binary values
    return np.array(trace_data)  # Return the captured data

# Integrate trace capture in the loop
for i in tnrange(N, desc='Capturing traces'):
    key, text = ktp.next()  # Generate the next key and plaintext pair
    textin.append(text)
    keys.append(key)
    
    # Encrypt using the CW305 target
    ciphertext = cw.encrypt(target, text, key)  # Note: capture_trace should be called after setting key and plaintext
    cipher = AES.new(bytes(key), AES.MODE_ECB)  # Initialize cipher for verification
    expected_ciphertext = cipher.encrypt(bytes(text))
    assert list(ciphertext) == list(expected_ciphertext), \
        f"Incorrect encryption result! Got {ciphertext} Exp {list(expected_ciphertext)}"
    
    # Capture power trace
    trace = capture_trace()  # Capture the power trace using the oscilloscope
    traces.append(trace)  # Append the captured trace to the list of traces
    project.traces.append(cw.Trace(trace, text, ciphertext, key))  # Save trace to project

project.save()

Capturing traces:   0%|          | 0/1000 [00:00<?, ?it/s]

AssertionError: Incorrect encryption result! Got bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00') Exp [44, 169, 241, 9, 192, 204, 136, 218, 71, 208, 52, 236, 133, 221, 218, 219]